# DS1: Finding the appropriate $\Delta t$ for a given precision

In [ ]:
from pylab import *

## Differentiating a function

In this part of the code, we will differentiate a function (in this case, the distance) to find its derivative (the velocity). In the cell below, I define the distance function: it accepts two parameters, `a` and `t`, and returns a value of `0.5*a*t**2`.

In [ ]:
def distance(a,t):
    s = 0.5*a*t**2
    return s

Once the distance function has been defined, I need to write the code to find the value of $\Delta t$ that would keep its derivative (the velocity) constant up to some precision `prec`. 

We know from the definition of the derivative, that $$v_{\Delta t} (t) = \frac{s(t+\Delta t) - s(t)}{\Delta t},$$

where I have indicated a subscript $\Delta t$ in the velocity to remind you that the value of $v(t)$ using this defintion depends on the choice of $\Delta t$ in general. We need to find that $\Delta t$ such that $v_{\Delta t}$ remains constant (and is therefore *independent* of $\Delta t$) for all smaller values of $\Delta t$.

In order to do this, I define a *function* `diff_req_dt` which accepts two parameters, the time at which I want the derivative `t`, and the precision up to which I want it `prec`.

In [ ]:
def diff_req_dt(t,prec):
    deltaT=1            # Begin with a large value of deltaT
    
    a = 9.8             # The value of the acceleration. 
    old_v = 0           # A temporary variable to hold the `old' value of
                        # the velocity. You'll see how it's used below. This
                        # line just gives it an initial (random) value.
    
    v=0                 # Initial (random) value for v.
    
    while(True):        # This is called a `while` loop. It will
                        # continue to run so long as the condition
                        # in the brackets is satisfied. In this case
                        # the condition (True) is *always* satsisfied
                        # and so it runs forever, or until it hits a
                        # `break` command.
            
        v = round((distance(a,t+deltaT) - distance(a,t))/deltaT,prec)
        
        deltaT = deltaT*0.1 # Make deltaT ten times smaller than it used to be.
        
        if(old_v==v):   # If the current value of v is the same as the one from
                        # the previous iteration of the loop.
            
            break       # End the loop
            
        old_v = v       # Replace the value of old_v with the current value of v.
                        # Imp: this line is not run if the loop breaks.
    
    return 10*deltaT    # After everything is done, return the *previous* value
                        # of deltaT (which is 10 times the current deltaT).

I can now *call* this function with a given value of `t` and `prec`, and it will return the value of `deltaT`. You can run the following cell to find the value of $\Delta t$ required for the velocity at $t=7$, upto a precision of 3 decimal places.

In [ ]:
diff_req_dt(7,3)        # Run this cell to get the deltaT required
                        # at t=7 with precision 3.

## Integrating a function

Suppose now that you are instead *given* the velocity function and asked to find the distance. From school, you must know that you would need to integrate the velocity to find the position. This is now what we will do in this part of the code: given a function (in this case, the velocity) we will find its integral (the distance). In the cell below, I define the velocity function: it accepts two parameters, `a` and `t`, and returns a value of `a*t`.

In [ ]:
def velocity(a,t):
    v = a*t
    return v

#### Forward Integration

We now need, as before, to write the code that finds the value of $\Delta t$ that will keep the integral (the distance) constant up to some precision `prec`.

Integration is slightly less intuitive than differentiation, though not by much. As mentioned in class, the distance at any time $T$ can be obtained by the formula

$$s_{\Delta t}(T) = s(0) + v\,(\Delta t) \Delta t + v\,(2\times\Delta t) \Delta t + \dots = s(0) + \sum_{i=1}^{N} v\,\big( (i-1) \Delta t\big) \Delta t,$$

where as before the subscript $\Delta t$ indicates that using this defintion, the value of $s$ depends in general on the choice of $\Delta t$. The number $N$ is the number of steps that I have broken up the interval of time from $t=0$ to $t=T$, i.e. $T = N\Delta t$.

So what would my method be? I would first need to start with an initial value of $s(0)$, say $s(0)=0$. Then, **loop** over successive values of $i$ from 1 to $N$, at each step increasing $s$ by $v\,\big( (i-1) \Delta t\big) \Delta t$. Once I have finished $N$ steps, I have $s_{\Delta t}(T)$.

Now, I repeat this process for a different value of $\Delta t$. If I make $\Delta t$ *smaller*, then I have made $N$ larger. And I repeat the loop I mentioned above for this new value of $N$, and so on until $s_{\Delta t}(T)$ does not change up to the precision required.

It should be clear from this explanation that we require **two loops**: one to keep running for different values of $\Delta t$, and one to loop from $i$ to $N$ inside it. In order to do this, I define a *function* `int_req_dt` which accepts two parameters, the time at which I want the integral `t`, and the precision up to which I want it `prec`.

In [ ]:
def int_req_dt(t,prec):
    deltaT=1            # Begin with a large value of deltaT
    
    a = 9.8             # The value of the acceleration. 
    old_s = 1           # A temporary variable to hold the `old' value of
                        # the velocity. You'll see how it's used below. This
                        # line just gives it an initial (random) value.
    
    s = 0               # The initial value of $s(0)$.
    
    
    while(True):                # The first loop, which changes deltaT
        deltaT=deltaT*0.1       # Reduce deltaT by a factor of 10.
        
        N = int(t/deltaT)+1     # This is the total number of steps.
        
        s=0                     # Every time this loop runs, the original
                                # distance is reset to 0, as it is a new run.
        
        for i in range(1,N+1):  # The second loop, over i.
            t1 = (i-1)*deltaT   # The definition of the total time.
            v = velocity(a,t1)  # Finding the velocity at that time.
            deltaS = v*deltaT   # Finding the change in position due to this velocity
            s = s + deltaS      # Incrementing the position.

        s=round(s,prec)         # The distance is rounded up to `prec`.

        if(old_s == s):         # As before, if this calculated s is the
                                # same as s from the previous iteration,
            break               # Stop everything!
        old_s = s               # Otherwise replace the old value with this value
                                # and continue the outer loop.
        
    return deltaT*10            # After the loop ends, return the previous value of deltaT.

#### Mid-point integration

We can now attempt a method that is slightly more efficient. In the previous case, we assumed that the velocity $v$ did not change "much" during the time interval $\Delta t$. As a result, we used the value of $v(t)$ to represent the velocity between time $t$ and $t+\Delta t$. Intuitively, you might guess that a better approximation would be if we used the value $v(t+\Delta t/2)$ to represent the velocity between time $t$ and $t+\Delta t$, which is what we will be using in the `midint_req_dt` function below. The rest of the function is the same.

In [ ]:
def midint_req_dt(t,prec):
    deltaT=1
    
    a=9.8
    old_s = 1
    s = 0 
    
    while(True):
        deltaT=deltaT*0.1
        
        N = int(t/deltaT)+1
        
        s=0
        
        for i in range(1,N+1):
            t1 = (i-1)*deltaT
            v = velocity(a,t1+deltaT/2)   # Here we use deltaT/2 instead of deltaT.
            deltaS = v*deltaT
            s = s + deltaS

        s=round(s,prec)

        if(old_s == s):
            break   
        old_s = s
        
    return deltaT*10

## User input

This part is just a bit of fun. We're going to ask the user to input the value of `prec` and print out the $\Delta t$ required if we used forward integration and if we used mid-point integration. (We've chosen to fix `t=10` but you could ask the user for that as well.)

In [ ]:
precision = int(input("Enter the required precision: "))

time = 10

print("The required deltaT by midpoint integration for a precision of ", precision, " is ",midint_req_dt(time,precision))
print("The required deltaT by forward integration for a precision of ", precision, " is ",int_req_dt(time,precision))


#### Note:
You should see that the midpoint integration method is *much* more efficient than the forward integration method! 

You can now improve on this program: you could, for example, get it to actually return the value of the calculated integral. You could also get it to return the time that it took, so that you could objectively compare the two methods to find out which you'd rather use. I leave these as exercises for you to do.